In [1]:
# carregando sistema físico
from m3l.structure import System as sys
import json
#
system = sys()
system.loadSystem('system_new.json')
system.convertUnits()
#system.save('system_au.json')
#system.convertUnitsInv()
#system.atoms
#system.cell

In [2]:
system.ACONV

array(1.88972613)

In [3]:
# definindo o modelo de interação entre os átomos (campo de força)
from m3l.molecular_dynamics import ForceField
class Forces(ForceField):
    def __init__(self):
        super().__init__()
        self.parameters((0.2385*self.ECONV, 3.405*self.ACONV, 10.0*self.ACONV))

model = Forces()
model()

array([3.80200013e-04, 6.43451746e+00, 1.88972613e+01])

In [4]:
# definindo o modelo estatístico (ensemble)
from m3l.molecular_dynamics import Ensemble as en
#
force_field = model()
ensemble = en(temp_bath = 100.0, press_bath = 1000.0, timestep = 1.0e-4, force_field = force_field, tstat = 1.0, pstat = 1.0, bfactor = 5.0e-5)
ensemble(system)

In [ ]:
# executando looping
import time
import csv
import numpy as np
#
n_steps = 100000
i_step = 0
dtim = []
start = time.time()
with open('history.csv', 'w', newline = '') as csvfile1, open('thermodynamics.csv', 'w', newline = '') as csvfile2:
    fieldnames1 = ['time', 'atom', 'x', 'y', 'z']
    writer1 = csv.DictWriter(csvfile1, fieldnames = fieldnames1)
    writer1.writeheader()
    fieldnames2 = ['time', 'volume', 'temperature', 'epotential', 'pressure']
    writer2 = csv.DictWriter(csvfile2, fieldnames = fieldnames2)
    writer2.writeheader()
    for step in range(n_steps):
        system = ensemble(system)
        if step%100 == 0:
            dtim.append(ensemble.dtimestep/system.TIMECONV)
            i_step += 1
            writer2.writerow({
                'time': ensemble.dtimestep*step/system.TIMECONV, 
                'volume': np.prod(system.cell)/system.ACONV**3,
                'temperature': system.temperature/system.TEMPCONV,
                'epotential': system.epotential/system.ECONV,
                'pressure': system.pressure/system.PCONV})
            for atom in system.atoms:
                writer1.writerow({
                    'time': ensemble.dtimestep*step/system.TIMECONV,
                    'atom': atom[0],
                    'x': atom[1]/system.ACONV,
                    'y': atom[2]/system.ACONV,
                    'z': atom[3]/system.ACONV})
            #print(f'Step: {i_step}; Energy: {system.epotential*system.ECONV}')
#
end = time.time()
print(f'Elapsed time: {end - start}')

In [ ]:
#import matplotlib.pyplot as plt
#import numpy as np
#dataset = np.loadtxt("thermodynamics.csv", dtype=np.float32, delimiter = ",", skiprows=1)
#x_list = [item[0].item() for item in dataset]
#y_list = [(item[3].item()) for item in dataset]
#plt.plot(x_list, y_list)
#plt.ylabel('Energia')
#plt.ylim(-1,1000)
#plt.plot(range(len(dtim)), dtim)
#plt.ylabel('Timestep')
#plt.show()

In [ ]:
#import matplotlib.pyplot as plt
#import numpy as np
#dataset = np.loadtxt("thermodynamics.csv", dtype=np.float32, delimiter = ",", skiprows=1)
#x_list = [item[0].item() for item in dataset]
#y_list = [item[1].item() for item in dataset]
#plt.plot(x_list, y_list)
#plt.ylabel('Temperatura')
#plt.ylim(80,90)
#plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
dataset = np.loadtxt("thermodynamics.csv", dtype=np.float32, delimiter = ",", skiprows=1)
time = [item[0].item() for item in dataset]
vol = [item[1].item() for item in dataset]
temp = [item[2].item() for item in dataset]
epot = [item[3].item() for item in dataset]
press = [item[4].item() for item in dataset]
#plt.plot(time, temp)
fig, axs = plt.subplots(2, 2)
axs[0, 0].plot(time, vol)
axs[0, 0].set_title("Volume (A^3)")
axs[0, 1].plot(time, temp)
axs[0, 1].set_title("Temperatura (K)")
axs[1, 0].plot(time, epot)
axs[1, 0].set_title("Energia Potencial (kcal/mol)")
axs[1, 1].plot(time, press)
axs[1, 1].set_title("Pressão (atm)")
fig.tight_layout()
for ax in axs.flat:
    ax.set(xlabel='Tempo (ps)')

# Hide x labels and tick labels for top plots and y ticks for right plots.
#for ax in axs.flat:
#    ax.label_outer()
plt.savefig("MD.jpg")

In [ ]:
system.convertUnitsInv()
system.save('system_new.json')